In [6]:
import numpy as np
from pydgrid.pydgrid import grid
from pydgrid.pf import pf_eval,time_serie

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import push_notebook
output_notebook()

Loading BokehJS ...

## 907 bus 3 wire system with transformer

### Execute power flow 

In [7]:
sys1 = grid()
sys1.read('n1_f1.json')  # Load data
sys1.read_loads_shapes('n1_f1_load_shapes.json')  
sys1.pf_solver = 2
sys1.pf()  # solve power flow
        
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

In [8]:
%timeit sys1.pf()  # solve power flow

1.85 ms ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Graph with obtained results

In [9]:
sys1.s_radio_scale =0.5
sys1.s_radio_min =5
sys1.s_radio_max =100
sys1.snapshot(60*60*12)
sys1.pf() 
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents
sys1.bokeh_tools()

p_grid = figure(width=900, height=800,
           title='3 bus 4 wire system with transformer') 

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p_grid.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p_grid.circle(source=source, x='x', y='y', size='s_radio', color="s_color", alpha=0.5)

p_grid.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p_grid.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update_grid(t_h=0.0):
    sys1.snapshot(60*60*t_h)

    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p_grid, notebook_handle=True)



In [5]:
from ipywidgets import interact
interact(update_grid, t_h=(0,24, 0.01))

A Jupyter Widget

<function __main__.update_grid>

### Interaction 

In [ ]:
p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           y_range = [0.95,1.01], #x_range = [50,-300], 
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
#cr = p1.circle(source=source, x='y', y='v_an_pu', size=15, color="red", alpha=0.5)
p.circle(source=source, x='x', y='v_an_pu', size=15, color="red", alpha=0.5)
p.circle(source=source, x='x', y='v_bn_pu', size=15, color="green", alpha=0.5)
p.circle(source=source, x='x', y='v_cn_pu', size=15, color="blue", alpha=0.5)

#p1.circle(source=source, x='y', y='v_bn_pu', size=15, color="green", alpha=0.5)
#p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
#p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
#p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))

def update(t_h=0.0):
    sys1.snapshot(60*60*12)

    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [ ]:
from ipywidgets import interact
interact(update, t_h=(0,24, 0.01))

### Get element transfomers results

In [20]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numba
from scipy.interpolate import Rbf
#%matplotlib notebook


def plot_multi(sys1,i,t):
    
    # https://github.com/asher-pembroke/bokeh-tools
    #for t in np.linspace(0,24*3600,10):
    sys1.snapshot(t)

    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()


    P_a  = np.array([item['p_a'] for item in sys1.buses])
    P_b  = np.array([item['p_b'] for item in sys1.buses])
    P_c  = np.array([item['p_c'] for item in sys1.buses])
    P_t = P_a+P_b+P_c
    mask = abs(P_t)>1
    V_an  = np.array([item['v_an']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
    V_bn  = np.array([item['v_bn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
    V_cn  = np.array([item['v_cn']/item['U_kV']/1000*np.sqrt(3) for item in sys1.buses])
    Pos_x = np.array([item['pos_x'] for item in sys1.buses])
    Pos_y = np.array([item['pos_y'] for item in sys1.buses])
    xy = Pos_x[mask] + 1j*Pos_y[mask]
    u, indices = np.unique(xy, return_index=True)
    x_min = np.min(Pos_x)
    x_max = np.max(Pos_x)
    y_min = np.min(Pos_y)
    y_max = np.max(Pos_y)

    x_margin = 10
    y_margin = 10

    x_min2 = x_min-x_margin
    x_max2 = x_max+x_margin
    y_min2 = y_min-y_margin
    y_max2 = y_max+y_margin

    V_a_avg = np.average(V_an)*0+1
    V_b_avg = np.average(V_bn)*0+1
    V_c_avg = np.average(V_cn)*0+1
    V_inf = 0

    V_a_avg = 1
    V_b_avg = 1
    V_c_avg = 1

    Pos_x_e = np.hstack((x_min2, x_min2, Pos_x[mask][indices], x_max2, x_max2))
    Pos_y_e = np.hstack((y_min2, y_max2, Pos_y[mask][indices], y_max2, y_min2))
    V_an_e  = np.hstack((V_a_avg,V_a_avg, V_an[mask][indices], V_a_avg,V_a_avg))
    V_bn_e  = np.hstack((V_b_avg,V_b_avg, V_bn[mask][indices], V_b_avg,V_b_avg))
    V_cn_e  = np.hstack((V_c_avg,V_c_avg, V_cn[mask][indices], V_c_avg,V_c_avg))
    rbfi_a = Rbf(Pos_x_e, Pos_y_e, V_an_e - V_a_avg, function='linear')
    rbfi_b = Rbf(Pos_x_e, Pos_y_e, V_bn_e - V_b_avg, function='linear')
    rbfi_c = Rbf(Pos_x_e, Pos_y_e, V_cn_e - V_c_avg, function='linear')

    x =  np.linspace(x_min2,x_max2, 400)
    y =  np.linspace(y_min2,y_max2, 400)
    X, Y = np.meshgrid(x, y)
    Z_a = rbfi_a(X, Y)
    Z_b = rbfi_b(X, Y)
    Z_c = rbfi_c(X, Y)

    z_min = -0.005
    z_max =  0.005
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111) #, projection='3d')
    ax.pcolormesh(X, Y, Z_a, cmap = plt.get_cmap('rainbow'), vmin=z_min, vmax=z_max)


    fig.savefig('./animation/out_{:s}.png'.format(str(int(i)).zfill(3)))
    plt.close('all')

plot_multi(sys1,i,2)

In [21]:
import multiprocessing
# ffmpeg -f image2 -r 6 -pattern_type glob -i '*.png' output.mp4

def worker(sys1,t):
    """thread worker function"""
    plot_multi(sys1,i,t)
    return


jobs = []
for i in range(24):
    t = i*60*64
    p = multiprocessing.Process(target=plot_multi, args=(sys1,i,t))
    jobs.append(p)
    p.start()

In [ ]:
from bokeh.io import export_png
from bokeh.io import export_svgs

# must give a vector of image data for image parameter
p = figure(width=800, height=800,
           title='European LV Network (CIGRE)', x_range=[x_min2,x_max2], y_range=[y_min2, y_max2])

p.image(image=[Z_a], x=x_min2, y=y_min2, dw=(x_max2-x_min2), dh=(y_max2-y_min2), palette="Spectral11")

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size='s_radio', color="s_color", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


show(p)  # open a browser
#bokeh.io.export_png(p, filename=None, height=None, width=None, webdriver=None)
p.export_png()


In [ ]:
np.max(Z_a)


In [ ]:
fig = plt.figure()
ax = fig.gca()

CS = ax.(X,Y,Z_a)